<a href="https://colab.research.google.com/github/denin-thomas/P6-American-Sign-Language-Translation/blob/main/Sign_To_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Dataset.zip to Dataset.zip


In [14]:
import zipfile
import os

with zipfile.ZipFile("Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("Dataset")


In [15]:
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf

# Load training dataset (grayscale, smaller size)
training_dataset = image_dataset_from_directory(
    directory='Dataset/Dataset',
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',              # Use grayscale
    batch_size=32,
    image_size=(128, 128),               # Reduced image size
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    verbose=True
)

# Load validation dataset
validation_dataset = image_dataset_from_directory(
    directory='Dataset/Dataset',
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    verbose=True
)

# Normalize pixel values to [0, 1]
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

training_dataset = training_dataset.map(normalize)
validation_dataset = validation_dataset.map(normalize)





Found 4681 files belonging to 26 classes.
Using 3745 files for training.
Found 4681 files belonging to 26 classes.
Using 936 files for validation.


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras import optimizers

model = Sequential([
    Input(shape=(128, 128, 1)),  # Grayscale input (1 channel)

    # Block 1
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    # Block 2
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    # Block 3
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Flatten(),
    Dense(128, activation='relu'),  # Reduced size
    Dropout(0.3),
    Dense(26, activation='softmax')  # 26 classes
])

optimizer = optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(training_dataset,
                   epochs=10,
                   validation_data=validation_dataset)

Epoch 1/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 17s 69ms/step - accuracy: 0.1825 - loss: 3.1905 - val_accuracy: 0.0470 - val_loss: 3.3042
Epoch 2/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.5634 - loss: 1.4431 - val_accuracy: 0.1549 - val_loss: 3.0641
Epoch 3/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7719 - loss: 0.7644 - val_accuracy: 0.6090 - val_loss: 2.0861
Epoch 4/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.8413 - loss: 0.5701 - val_accuracy: 0.9167 - val_loss: 0.8191
Epoch 5/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.8838 - loss: 0.3807 - val_accuracy: 0.9626 - val_loss: 0.2998
Epoch 6/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9142 - loss: 0.2846 - val_accuracy: 0.9797 - val_loss: 0.1417
Epoch 7/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.9074 - loss: 0.3034 - val_accuracy: 0.9915 - val_loss: 0.0777
Epoch 8/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9424 - loss: 0.1846 - val_ac

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(optimizer=optimizers.Adam(learning_rate=0.00001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = model.fit(training_dataset,
                   epochs=25,
                    initial_epoch=15,
                   validation_data=validation_dataset,
                    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)])


Epoch 16/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 40s 261ms/step - accuracy: 0.8094 - loss: 3.7049 - val_accuracy: 0.9840 - val_loss: 2.7567
Epoch 17/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 23s 195ms/step - accuracy: 0.9202 - loss: 2.9321 - val_accuracy: 0.9936 - val_loss: 2.4826
Epoch 18/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 41s 195ms/step - accuracy: 0.9483 - loss: 2.5669 - val_accuracy: 0.9947 - val_loss: 2.2555
Epoch 19/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 41s 198ms/step - accuracy: 0.9674 - loss: 2.3213 - val_accuracy: 0.9947 - val_loss: 2.0858
Epoch 20/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 41s 196ms/step - accuracy: 0.9816 - loss: 2.0944 - val_accuracy: 0.9968 - val_loss: 1.9293
Epoch 21/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 23s 198ms/step - accuracy: 0.9820 - loss: 1.9331 - val_accuracy: 0.9979 - val_loss: 1.7901
Epoch 22/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 42s 205ms/step - accuracy: 0.9889 - loss: 1.7774 - val_accuracy: 0.9979 - val_loss: 1.6592
Epoch 23/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 41s 209ms/step - accuracy: 0.9863 -

In [13]:
from google.colab import files

model.save('model.keras')
files.download('model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>